# Deep MNIST for Experts

### Dataset

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Session

In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()

### Parameters

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [5]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [6]:
# initialization
sess.run(tf.initialize_all_variables())

In [7]:
# prediction
y = tf.matmul(x,W) + b

In [8]:
# cost function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))

### Training

In [9]:
# param udates
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [10]:
for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

### Evaluation

In [11]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [12]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9188


### Multilayer Convolutional Network

In [14]:
# Weight Initialization

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [15]:
# Convolution and Pooling

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [16]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [17]:
x_image = tf.reshape(x, [-1,28,28,1])

In [18]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [19]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [21]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [22]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [24]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [25]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
                x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
        
        print("test accuracy %g"%accuracy.eval(feed_dict={
                    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.08
test accuracy 0.1053
step 100, training accuracy 0.08
test accuracy 0.1279
step 200, training accuracy 0.06
test accuracy 0.1449
step 300, training accuracy 0.12
test accuracy 0.1622
step 400, training accuracy 0.1
test accuracy 0.1918
step 500, training accuracy 0.22
test accuracy 0.1822
step 600, training accuracy 0.16
test accuracy 0.1569
step 700, training accuracy 0.22
test accuracy 0.1487
step 800, training accuracy 0.1
test accuracy 0.1389
step 900, training accuracy 0.12
test accuracy 0.1473
step 1000, training accuracy 0.2
test accuracy 0.1737
step 1100, training accuracy 0.16
test accuracy 0.1988
step 1200, training accuracy 0.24
test accuracy 0.2227
step 1300, training accuracy 0.14
test accuracy 0.2445
step 1400, training accuracy 0.36
test accuracy 0.2627
step 1500, training accuracy 0.26
test accuracy 0.2792
step 1600, training accuracy 0.3
test accuracy 0.2835
step 1700, training accuracy 0.38
test accuracy 0.2975
step 1800, training accura

KeyboardInterrupt: 